# UdaciHeadline: LLM Inference Optimization Project

## Project Introduction
Large Language Models (LLMs) are transforming content creation, but deploying them efficiently remains a major hurdle. Imagine you're an ML Engineer at a bustling online news portal. Your key task? Automatically generating catchy headlines from article summaries using an LLM. The problem? The current inference process is sluggish, causing publication delays and driving up operational costs. In this project, UdaciHeadline, you'll step into this role and tackle this critical challenge head-on. Your mission is to accelerate the headline generation pipeline significantly by applying state-of-the-art LLM inference optimization techniques. Get ready to dive deep into practical optimization and deployment!

## Project Summary
This project provides hands-on experience in optimizing the inference performance of a pre-trained Large Language Model (like Llama-3.2-1B) for news headline generation. You will bring together concepts of LLM architecture, optimization techniques, and deployment frameworks. Specifically, you will:

1.  **Establish a baseline** inference pipeline and profile its performance.
2.  Implement and evaluate architectural optimizations like **KV-caching**.
3.  Apply model compression techniques like **quantization** and **pruning**.
4.  Configure and benchmark **distributed inference** using Tensor and Pipeline Parallelism.
5.  Apply advanced decoding mechanisms like **speculative decoding**.
6.  Perform comprehensive **benchmarking and analysis** across all stages.
7.  Produce a **final report** summarizing findings and trade-offs.

## Imports and Global Configuration

Let's import the libraries we'll use throughout the project and define some constants like the model name and the prompt template.

In [67]:
import os
import torch
import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from evaluate import load as load_metric
from time import time
from pprint import pprint
import torch.nn.utils.prune as prune

os.environ["HF_HUB_OFFLINE"] = "1"
# ---- Constants ----
MODEL_NAME = "meta-llama/Meta-Llama-3-1B"
MAX_NEW_TOKENS = 0 # Max length for the generated headline

PROMPT = \
"""
TODO: iterate on prompts to ensure good quality output.
"""

## Data Loading

We will use the "News Category Dataset" from Kaggle. The `kagglehub` library makes it easy to download and access. Your task is to implement the function to load and preprocess the data according to the docstring.

In [68]:
ls ../../code/dataset

News_Category_Dataset.json*


In [69]:

def load_news_dataset(path):
    """TODO: Implement the data loading and preprocessing logic here."""
    dataset = load_dataset("json", data_files=path, split="train[:1000]")
    articles = [item["short_description"] for item in dataset]

    return dataset,articles


# 2. Baseline Performance

Before we can optimize, we need a starting point. Here, you'll establish the baseline performance of the `Llama-3.2-1B` model without any specific optimizations. We will measure latency, throughput, and the quality of the generated headlines using the ROUGE score.

### Your Task: Implement the Evaluation Pipeline
You need to implement the core functions for loading a model, generating a headline, and evaluating performance. These functions will be reused for every optimization technique.

In [70]:
def load_model(model_name, quantization_config=None):
    """TODO: Implement the logic for loading a tokenizer and model."""
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).cuda()
    model.eval()

    return model

def generate_headline(model, tokenizer, summary, generation_args):
    """TODO: Implement the headline generation and latency measurement logic."""
    eadlines = []
    latencies = []
    with torch.no_grad():
        for text in texts:
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to("cuda")
            start = time.time()
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_length,
                use_cache=False,  # Baseline: no KV caching
                do_sample=False
            )
            end = time.time()
            latencies.append(end - start)
            headline = tokenizer.decode(outputs[0], skip_special_tokens=True)
            headlines.append(headline)
    return headlines, latencies

def report_metrics(times):#results, latencies, max_new_tokens):
    """TODO: Implement the logic for calculating and reporting all performance metrics."""
    avg_latency = sum(times) / len(times)
    p99_latency = sorted(times)[int(0.99 * len(times))]
    throughput = len(times) / sum(times)
    gpu_memory = torch.cuda.max_memory_allocated() / (1024 ** 2)
    print(f"Avg Latency: {avg_latency:.3f}s")
    print(f"P99 Latency: {p99_latency:.3f}s")
    print(f"Throughput: {throughput:.2f} samples/sec")
    print(f"Max GPU Memory: {gpu_memory:.2f} MB")

    pass

def evaluate_model(dataset, model, tokenizer, generation_args, n=20):
    """TODO: Implement the model evaluation loop."""
    rouge = load_metric("rouge")
    references = [item["headline"] for item in dataset[:100]]
    results = rouge.compute(predictions=generated, references=references)
    print("ROUGE Scores:", results)
    
    return results

In [88]:
ls /voc/shared/models/llama/

Llama-3.2-1B/  Llama-3.3-70B-Instruct/
Llama-3.2-3B/  Llama-4-Scout-17B-16E-Instruct/


In [89]:
# TODO: Establish your baseline performance
datasets, articles = load_news_dataset("News_Category_Dataset.json")
model = load_model("/voc/shared/models/llama/Llama-3.2-1B")
sample_texts = articles[:100]
generated, times = generate_headlines(sample_texts)
report_metrics

/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


RuntimeError: CUDA driver initialization failed, you might not have a CUDA gpu.

# 3. Architectural Optimization: KV Caching

**Your Task:** One of the most effective ways to speed up token generation is using a Key-Value (KV) cache. This avoids re-computing attention scores for tokens that are already part of the sequence. Enable the `use_cache` flag in the generation arguments and re-run the evaluation. Observe the impact on latency and throughput.

In [ ]:
# TODO: Evaluate the model with KV Caching enabled.

# 4. Model Compression: Pruning

**Your Task:** Pruning removes redundant model weights, which can reduce model size and potentially speed up inference. Here, you will implement unstructured, magnitude-based pruning by creating a function that applies it to the model's linear layers and then evaluating the result.

In [ ]:
def prune_model_weights(model, amount=0.3):
    """TODO: Applies L1 unstructured pruning to the linear layers of a model."""
    pass

# TODO: Evaluate the pruned model.

# 5. Model Compression: Quantization

**Your Task:** Quantization reduces the precision of model weights (e.g., from 16-bit to 4-bit), significantly cutting down memory usage and often speeding up inference. You will define a 4-bit quantization configuration and use it to load and evaluate a new model.

In [ ]:
# TODO: Implement and evaluate 4-bit quantization.

# 6. Distributed Inference (Multi-GPU)

**Your Task:** If you have multiple GPUs, you can split the model across them to reduce the memory burden on a single GPU and potentially improve latency. We will explore two common techniques: Tensor Parallelism and Pipeline Parallelism.

*Note: This section requires a multi-GPU environment.*

### Tensor Parallelism
Tensor parallelism splits individual model layers (the tensors) across multiple GPUs. Operations like matrix multiplications are executed in parallel on different GPUs, and the results are aggregated. This is highly effective for reducing the memory footprint of very large layers. The `accelerate` library can handle this automatically via `device_map="auto"`.

### Pipeline Parallelism
Pipeline parallelism assigns entire layers or blocks of layers to different GPUs, creating a sequence or "pipeline" that the data flows through. For example, layers 1-10 run on GPU 0, layers 11-20 run on GPU 1, and so on. This is useful for very deep models where even a single layer might be too large for one GPU after tensor parallelism.

In [ ]:
# TODO: Check for multi-GPU environment and evaluate with Tensor Parallelism.
# The `device_map="auto"` in your `load_model` function should automatically apply this.

In [ ]:
# TODO: Evaluate with Pipeline Parallelism.
# This is more advanced and may require manually defining a device_map to assign
# different layers of the model to different GPUs.

# 7. Advanced Decoding: Speculative Decoding

**Your Task:** Speculative decoding uses a smaller, faster "draft" model to generate several candidate tokens. A larger, more accurate "target" model then verifies these tokens in a single forward pass. This can significantly speed up generation if the draft model is a good predictor. You will load a larger target model and a smaller draft model, benchmark the target model alone, and then benchmark it with assistance from the draft model.

In [ ]:
# TODO: Implement and evaluate speculative decoding.

# 8. Final Report and Analysis

**Your Task:** Consolidate your findings into a summary report. 

1.  Fill in the Markdown table below with the **Latency**, **Throughput**, and **ROUGE scores** for each optimization technique you implemented.
2. Compile the final Project Report in PDF format:
    *   Document the entire process, detailing the methodology, techniques, and libraries used.
    *   Present the final benchmark results clearly.
    *   Provide a thorough analysis of the trade-offs between performance, resources, and quality for each optimization step.
    *   Conclude with recommendations for the most effective optimization strategy for this specific headline generation task, supported by your data.

Some example questions for discussing the trade-offs:
    *   Which method gave the best performance improvement?
    *   Did any methods significantly hurt the ROUGE score (quality)?
    *   Which optimization would you recommend for deployment in a production environment at the news portal, and why? Consider factors like cost, complexity, and performance.

## Performance Comparison

| Optimization Technique | Mean Latency (s) | Throughput (tokens/s) | ROUGE-1 Score |
|--------------------------|------------------|-----------------------|---------------|
| Baseline (No Cache)      | TODO             | TODO                  | TODO          |
| KV Caching               | TODO             | TODO                  | TODO          |
| Pruning (30%)            | TODO             | TODO                  | TODO          |
| Quantization (4-bit)     | TODO             | TODO                  | TODO          |
| Tensor Parallelism       | TODO             | TODO                  | TODO          |
| Pipeline Parallelism     | TODO             | TODO                  | TODO          |
| Speculative Decoding     | TODO             | TODO                  | TODO          |

---

